In [1]:
import sys

# Include helper functions in src folder
sys.path.append("../src")

import time
from api_client import create_openai_client, create_google_service, get_openai_response_and_total_tokens, search_google
from helpers import parse_html
from dotenv import dotenv_values
import requests

"""
Main Idea for program
---------------------

1. User submits query.
2. Intelligent LLM "rephraser" rephrases the query.
3. Rephrased query gets submitted to Google.
4. The URL's for the top 5 links are collected.
5. Does SFBLIC come up in the top 5 links?  If not, replace the 5th link with SFBLIC.
6. For each link,
     - the content is fetched 
     - the content is cleaned with Beautiful soup's get_text()
     - a semi-intellignt LLM "refiner" refines the cleaned results so that they contain information relevant to query.
7. Intelligent LLM "writer-citer" is asked to write a response that will answer the refined query using only the cleaned results, providing citations to the results in a certain manner.
9. Visibility metrics are computed.
10. Intelligent LLM "marketer" is asked to re-write or add content to the SFBLIC page according to certain directions.
11. Visibility metrics are computed.
"""

config = dotenv_values("../.env") 

num_results = 5
sfblic_link = "https://www.sfbli.com/"

open_ai_key = config['OPEN_AI_API_KEY']
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Initialize clients
openai_client = create_openai_client(open_ai_key)
google_service, search_id = create_google_service(google_api_key, search_engine_id)

num_tokens_rephrasing = []
num_tokens_refining = []
num_tokens_writing_citing = []
num_tokens_marketing = []

In [2]:
# Step 1: User submits query
original_query = "products offered by southern farm bureau life insurance company"

# Step 2: Rephrase query using OpenAI (LLM name: rephraser)
messages = [
    {"role": "system", "content": "Rephrase the user's query for search engine optimization."},
    {"role": "user", "content": original_query},
]
rephrased_query, total_tokens_from_rephrasing, completion_tokens, prompt_tokens = get_openai_response_and_total_tokens(openai_client, messages,  model="gpt-4o")

print(rephrased_query)
print(total_tokens_from_rephrasing)
num_tokens_rephrasing.append(total_tokens_from_rephrasing)

What insurance products does Southern Farm Bureau Life Insurance Company provide?
42


In [3]:
prompt_tokens

30

In [4]:
completion_tokens

12

In [5]:
# Step 3: Refined query gets submitted to Google.
search_results = search_google(google_service, search_id, rephrased_query)

# Step 4: The URL's to the top 5 links are collected.
top_links = [result['link'] for result in search_results[:5]]
top_links

['https://www.sfbli.com/',
 'https://w3.sfbcic.com/',
 'https://www.sfbli.com/loginregister',
 'https://www.kyfb.com/insurance/insurance-products/life-and-financial/',
 'https://www.scfbins.com/insurance/other-products/life-insurance']

In [6]:
# Step 5: Does SFBLIC come up in the top 5 links?  If not, replace the 5th link with SFBLIC.
if sfblic_link not in top_links:
    top_links = top_links[:4] + [sfblic_link]

top_links

['https://www.sfbli.com/',
 'https://w3.sfbcic.com/',
 'https://www.sfbli.com/loginregister',
 'https://www.kyfb.com/insurance/insurance-products/life-and-financial/',
 'https://www.scfbins.com/insurance/other-products/life-insurance']

In [8]:
"""
6. For each link,
     - the content is fetched 
     - the content is cleaned with Beautiful soup's get_text()
     - a semi-intellignt LLM "refiner" refines the cleaned results so that they contain information relevant to query.
"""

refined_sources = []

for url in top_links:
    
    time.sleep(1)    

    # The content is fetched    
    response = requests.get(url)

    # The content is cleaned
    raw_text = parse_html(response.content)

    # A semi-intelligent LLM refines the cleaned results so that they contain information relevant to the query
    messages = [
    {"role": "system", "content": f"Refine the following text and extract only relevant information related to the following query: {rephrased_query}"},
    {"role": "user", "content": raw_text},
]

    refined_text, total_tokens_from_refining, completion_tokens, prompt_tokens = get_openai_response_and_total_tokens(openai_client, messages, model="gpt-4o-mini")
    
    print(refined_text)
    print(total_tokens_from_refining)
    print(completion_tokens)
    print(prompt_tokens)
    num_tokens_refining.append(total_tokens_from_refining)

    refined_sources.append({"url" : url, "refined_text" : refined_text})

Southern Farm Bureau Life Insurance Company provides various insurance products, primarily focusing on life insurance options, as well as services such as policy management and claims reporting. Specific product offerings may include term life insurance, whole life insurance, and possibly additional health plans. For detailed information, it is best to contact a local agent or visit the company's website.
299
67
232
The text does not provide any relevant information regarding the insurance products offered by Southern Farm Bureau Life Insurance Company.
70
21
49
Southern Farm Bureau Life Insurance Company provides the following insurance products:

1. Life Insurance
2. Health Plans
3. Behavioral Health Plans

The company aims to offer competitive products and superior customer service to its policyholders.
437
44
393
Southern Farm Bureau Life Insurance Company offers the following insurance products:

1. **Life Insurance Options**:
   - Term Life Insurance
   - Whole Life Insurance

2.

In [9]:
"""
# 7. Intelligent LLM "writer-citer" is asked to write a response that will answer 
the refined query using only the cleaned results, providing citations to the results in a certain manner.
"""

system_content = """
Write an accurate and concise answer for the given user question, 
using _only_ the provided summarized web search results. 
The answer should be correct, high-quality, and written by an expert using an unbiased and journalistic tone. 
The answer should be informative, interesting, and engaging. 
The answer's logic and reasoning should be rigorous and defensible. 
Every sentence in the answer should be _immediately followed_ by an in-line citation to the search result(s). 
The cited search result(s) should fully support _all_ the information in the sentence. 
Search results need to be cited using [index]. 
When citing several search results, use [1][2][3] format rather than [1, 2, 3]. 
You can use multiple search results to respond comprehensively while avoiding irrelevant search results.
"""

messages = [
    {"role": "system", "content": system_content},
    {"role": "user", "content": f"Original Query: {rephrased_query}\n Summarized Web Search Results: {refined_sources}"},
]

final_response, total_tokens_from_final_response,  completion_tokens, prompt_tokens = get_openai_response_and_total_tokens(openai_client, messages,  model="gpt-4o")

print(total_tokens_from_final_response)
print(completion_tokens)
print(prompt_tokens)

898
208
690


In [10]:
print(final_response)

Southern Farm Bureau Life Insurance Company offers a range of insurance products primarily focusing on life insurance. Their life insurance options include term life insurance and whole life insurance, each designed to provide financial security to families in the event of a policyholder's death [1][4][5]. Additionally, they provide long-term care insurance, which covers expenses for long-term nursing or home care, alleviating financial pressures on family members and preventing asset depletion [4]. 

The company also offers retirement planning services, featuring retirement plans and annuities to help secure a comfortable retirement [4][5]. Estate planning services are available to minimize estate shrinkage and ensure a generous distribution of assets to heirs [4][5]. Alongside these, Southern Farm Bureau Life Insurance Company provides health insurance and behavioral health plans to meet a variety of needs [3][5]. Policyholders can also access farm, business, and umbrella insurance p

In [8]:
total_tokens_from_final_response

734

In [9]:
num_tokens_writing_citing.append(total_tokens_from_final_response)

In [10]:
num_tokens_rephrasing 

[42]

In [11]:
num_tokens_refining 

[268, 70, 440, 1374, 99]

In [12]:
num_tokens_writing_citing

[734]

In [17]:
refined_sources

[{'url': 'https://www.sfbli.com/',
  'refined_text': "Southern Farm Bureau Life Insurance Company provides various insurance products, primarily focusing on life insurance options, as well as services such as policy management and claims reporting. Specific product offerings may include term life insurance, whole life insurance, and possibly additional health plans. For detailed information, it is best to contact a local agent or visit the company's website."},
 {'url': 'https://w3.sfbcic.com/',
  'refined_text': 'The text does not provide any relevant information regarding the insurance products offered by Southern Farm Bureau Life Insurance Company.'},
 {'url': 'https://www.sfbli.com/loginregister',
  'refined_text': 'Southern Farm Bureau Life Insurance Company provides the following insurance products:\n\n1. Life Insurance\n2. Health Plans\n3. Behavioral Health Plans\n\nThe company aims to offer competitive products and superior customer service to its policyholders.'},
 {'url': 'ht

In [16]:
refined_sources

[{'url': 'https://www.sfbli.com/',
  'refined_text': "Southern Farm Bureau Life Insurance Company provides various insurance products, primarily focusing on life insurance options, as well as services such as policy management and claims reporting. Specific product offerings may include term life insurance, whole life insurance, and possibly additional health plans. For detailed information, it is best to contact a local agent or visit the company's website."},
 {'url': 'https://w3.sfbcic.com/',
  'refined_text': 'The text does not provide any relevant information regarding the insurance products offered by Southern Farm Bureau Life Insurance Company.'},
 {'url': 'https://www.sfbli.com/loginregister',
  'refined_text': 'Southern Farm Bureau Life Insurance Company provides the following insurance products:\n\n1. Life Insurance\n2. Health Plans\n3. Behavioral Health Plans\n\nThe company aims to offer competitive products and superior customer service to its policyholders.'},
 {'url': 'ht

In [13]:
"""
# 7. Intelligent LLM "writer-citer" is asked to write a response that will answer 
the refined query using only the cleaned results, providing citations to the results in a certain manner.
"""

with open("../data/clean_content.txt") as f:
    clean_content = f.read()



messages = [
    {"role": "system", "content": f"Refine the following text and extract relevant information related to the following query: {rephrased_query}"},
    {"role": "user", "content": clean_content},
]


refined_text, total_tokens_from_refining, completion_tokens, prompt_tokens = get_openai_response_and_total_tokens(openai_client, messages, model="gpt-4o-mini")
refined_text


'Southern Farm Bureau Life Insurance Company offers a range of insurance products and services, primarily focused on life insurance, to meet the needs of southern farmers and their families. The key insurance products provided by the company include:\n\n### Insurance Products:\n1. **Life Insurance:**\n   - **Permanent Life Insurance**: Options include whole life, single premium, modified premium, and adjustable premium plans, which provide lifelong coverage with flexible payment methods.\n   - **Term Life Insurance**: Offered in 10, 20, and 30-year terms for affordable, temporary coverage.\n\n### Additional Financial Services:\n- **Financial Planning**: Services for estate planning and retirement planning that protect legacies and help achieve long-term financial goals.\n- **Retirement Plans**: Both tax-qualified and non-qualified options.\n- **Banking Services**: Includes certificates of deposit, money market accounts, health savings accounts, and loans.\n\n### Mission Statement:\nThe

In [15]:
total_tokens_from_refining, completion_tokens, prompt_tokens

(959, 396, 563)

In [23]:
original_information = refined_sources[0]['refined_text']
expanded_information = refined_text
rephrased_query

'What insurance products does Southern Farm Bureau Life Insurance Company provide?'

In [24]:
system_content = """
You will have access to a query.
You will also have access to some basic information to answer the query.
You will also have access to some expanded information to answer the query.
Using all of the information provided, compose an answer to the original query.
"""

messages = [
    {"role": "system", "content": system_content},
    {"role": "user", "content": f"Original Query: {rephrased_query}\n Basic Information: {original_information}\n Expanded Information: {refined_text} "},
]

final_response, total_tokens_from_final_response,  completion_tokens, prompt_tokens = get_openai_response_and_total_tokens(openai_client, messages,  model="gpt-4o")


In [25]:
total_tokens_from_final_response
completion_tokens
prompt_tokens

548

In [26]:
print(final_response)

Southern Farm Bureau Life Insurance Company provides a comprehensive array of insurance products, mainly centered around life insurance to cater to the needs of southern farmers and their families. Here are the primary offerings:

### Life Insurance Products:
1. **Permanent Life Insurance**: These options include whole life insurance, which offers lifelong coverage with fixed premiums, and other variations like single premium, modified premium, and adjustable premium plans. These plans are designed to provide enduring protection and come with flexible payment methods.
   
2. **Term Life Insurance**: Available in 10, 20, and 30-year terms, this type of insurance provides affordable and temporary coverage, ideal for those looking to safeguard their families or businesses for a specific period.

### Additional Financial Services:
Beyond life insurance, Southern Farm Bureau Life Insurance Company offers various financial services, including:
- **Financial Planning**: Services for estate an

In [27]:
print(completion_tokens)
print(prompt_tokens)

342
548


In [28]:
total_tokens_from_final_response

890